# Oracle Graph + D3 visualization library

This sample notebook shows how to visualize a property graph coming from Oracle Graph Server using D3.js library using d3graph python module.


## 1. Connect to Oracle Graph and query property graph

First, connect to **Oracle Graph Server** using **Oracle Graph Client** for Python

In [1]:
import pypgx as pgx
import opg4py.graph_server as graph_server

# Disable default DEBUG logging
pgx.setloglevel("ROOT","ERROR")

In [2]:
base_url = "https://localhost:7007"
username = "soe"
password = "soe"

instance = graph_server.get_instance(base_url, username, password)
session = instance.create_session("python_pgx_client")
print(session)

PgxSession(id: 27cd7dc8-dbdb-4f75-9ad4-60e37898b0b0, name: python_pgx_client)


## 2. Read property graph

Get property graph from database to Oracle Graph Server

In [3]:
graph = session.read_graph_by_name('BANKPV', 'pg_view')
graph

PgxGraph(name: BANKPV, v: 1000000, e: 1005559, directed: True, memory(Mb): 202)

## 3. Run an algorithm

Execute an algorithm on the graph, for example PageRank


In [4]:
analyst=session.create_analyst()

In [5]:
analyst.pagerank(graph)

VertexProperty(name: pagerank, type: double, graph: BANKPV)

In [6]:
pgql="""SELECT e1.pagerank
FROM MATCH (e1) ON bankpv
order by e1.pagerank DESC
limit 10"""

rs=graph.query_pgql(pgql)
rs.to_pandas()

pagerank
0  0.000008
1  0.000008
2  0.000007
3  0.000007
4  0.000006
5  0.000006
6  0.000006
7  0.000005
8  0.000005
9  0.000005

## 4. Run query and get result for visualization

Run a PGQL query and put the information in some arrays

In [7]:
pgql="""SELECT id(v1),id(v2),x.amount
FROM MATCH ALL (e1) -[v]->{1,5} (e2) ON bankpv ONE ROW PER STEP ( v1, x, v2 )
WHERE e1.ID = 380 and e2.ID=786"""

rs=graph.query_pgql(pgql)

In [8]:
source = []
target =[]
weight = []

for i in rs:
    source.append(i[0])
    target.append(i[1])
    weight.append(i[2])
    #print (str(i[0]) + "-" + str(i[1]) + "-" + str(i[2]))

Make this graph public for other sessions.

In [9]:
graph.publish()
graph.pin()

## 5. Close session in Oracle Graph Server

For this example, the sessioin in Oracle Graph Server can be closed.

In [10]:
rs.close()
session.destroy()

## 6. Use d3graph to visualize the property graph

This is a convinient way to visualize the graph using D3.js without the need to deal with JavaScript or HTML.

In [11]:
# Import library
from d3graph import d3graph, vec2adjmat


# Create adjacency matrix
adjmat = vec2adjmat(source, target, weight=weight)


# Initialize
d3 = d3graph()

# Build force-directed graph with default settings
d3.graph(adjmat)
d3.set_edge_properties(directed=True)

d3.show(filepath='web/report.html',show_slider=False)

[d3graph] INFO> Set directed=True to see the markers!
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Converting adjacency matrix into source-target..
[d3graph] INFO> Number of unique nodes: 15
[d3graph] INFO> Keep only edges with weight>0
[d3graph] INFO> Converting source-target into adjacency matrix..
[d3graph] INFO> Making the matrix symmetric..
[d3graph] INFO> Slider range is set to [1925, 36744]
[d3graph] INFO> Write to path: [/opt/oracle/userhome/oracle/PYPGX/web/report.html]
[d3graph] INFO> File already exists and will be overwritten: [/opt/oracle/userhome/oracle/PYPGX/web/report.html]
